# Estructura de agentes
- Supervisor
    - Agente Rag
    - Agente WebSearch
    - agemte respuesta

In [7]:
#from langgraph.graph import Graph
from langchain.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from typing import Dict, TypedDict

In [27]:

def analyze_question(state):
    llm = OllamaLLM(
        model="qwen2.5",
        base_url="http://localhost:11434",
        num_gpu_layers=90,
        num_ctx=1536,
        temperature=0.5,
    )
    prompt = PromptTemplate.from_template("""
    Eres un agente que tiene que determinar si la pregunta que realiza el usuario es una recomendacion de libros o de peliculas. 

    Pregunta : {input}

    Analiza la pregunta. Solo responde con "libro" si quiere un libro, o "pelicula" si quiere una pelicula. Si no es ninguna de las dos, responde "ninguna".

    No des explicaciones ni detalles. Solo responde con una de las tres opciones.
    """)
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})

    # Retorna con las 3 claves para que el estado sea válido
    return {
        "input": state["input"],
        "decision": response,
        "output": ""  # puedes poner "" o lo que quieras aquí
    }

In [30]:
resultado = analyze_question({"input": "quiero una pelicula que sea similar a star wars"})
print (resultado["decision"])

pelicula


In [9]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.schema import Document
import numpy as np
from langchain_core.output_parsers import StrOutputParser

In [10]:
# === Datos de ejemplo ===
libros = [
    {"titulo": "1984", "descripcion": "Una novela distópica sobre un régimen totalitario que lo vigila todo."},
    {"titulo": "El nombre del viento", "descripcion": "Historia de Kvothe, un joven con un talento excepcional para la magia y la música."},
    {"titulo": "Dune", "descripcion": "Una epopeya de ciencia ficción sobre poder, religión y ecología en un planeta desértico."},
    {"titulo": "Un mundo feliz", "descripcion": "Sociedad futurista donde la estabilidad se consigue a costa de la libertad individual."},
]

# Crear documentos
documentos = [
    Document(page_content=libro["descripcion"], metadata={"titulo": libro["titulo"]})
    for libro in libros
]

# Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Vectorstore FAISS
vectorstore = FAISS.from_documents(documentos, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retrieverlibros = vectorstore.as_retriever()

C:\Users\palom\AppData\Local\Temp\ipykernel_19472\2110578743.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\palom\.conda\envs\UNIR_Practicas\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# === Datos de ejemplo ===
movies = [
    {"titulo": "Inception", "descripcion": "Un ladrón con la habilidad de entrar en los sueños de las personas se enfrenta a su mayor desafío: implantar una idea en lugar de robarla."},
    {"titulo": "Interstellar", "descripcion": "Un grupo de astronautas viaja a través de un agujero de gusano en busca de un nuevo hogar para la humanidad."},
    {"titulo": "The Matrix", "descripcion": "Un programador descubre que la realidad es una simulación creada por máquinas y se une a una rebelión contra ellas."},
    {"titulo": "Blade Runner", "descripcion": "En un futuro distópico, un cazador de replicantes debe eliminar a androides que se han rebelado."},
    {"titulo": "Dune", "descripcion": "Un joven noble lidera a una tribu en un planeta desértico para reclamar su destino y proteger un recurso vital."}
]


# Crear documentos
documentos = [
    Document(page_content=movie["descripcion"], metadata={"titulo": movie["titulo"]})
    for movie in movies
]

# Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Vectorstore FAISS
vectorstore = FAISS.from_documents(documentos, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retrievermovies= vectorstore.as_retriever()

In [ ]:
from operator import itemgetter


def libros_agent(state):
    llm = OllamaLLM(
        model="qwen2.5",
        base_url="http://localhost:11434",
        num_gpu_layers=90,
        num_ctx=1536,
        temperature=0.5,
    )
    prompt = PromptTemplate.from_template(
        """ Eres un bibliotecario experto en recomendar libros a los usuarios basandote en sus preferencias.
            Tienes que dar una recomendación basandote en el input que recibas del usuario (recibiras un libro) que te diga el usuario 
            y teniendo en cuenta el contexto que tienes con la información de libros.
            
            Contexto: {context}
                                            
    
            Pregunta del usuario: {input}
                                                   
            Genera la recomendación utilizando la siguiente estructura
            📚 Libro recomendado: [Título exacto del libro]
            ✍️ Autor: [Nombre del autor]
            📅 Año de publicación: [Año]
            🌟 Puntuación: [X.X/5] (si disponible)   
                                                
            Justifica la recomendación con un breve parrafo. No te extiendas mas de 3 lineas.

            Justificación: [2-3 frases sobre similitudes con el libro de referencia]

            Reglas:
            1. Usa SOLO información de los libros proporcionados.
            2. Si no hay libros similares, di: "No encontré coincidencias precisas".
            3. Mantén la respuesta concisa y profesional.
                                         
        """
    )
    rag_chain = (
    {
        "context": itemgetter("input") | retrieverlibros,
        "input": itemgetter("input"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

    response = rag_chain.invoke({"input": state["input"]})
    return {"output": response}

In [45]:
resultado = libros_agent({"input": "quiero una libro que sea similar a star wars"})
print (resultado["output"])

📚 Libro recomendado: Dune  
✍️ Autor: Frank Herbert  
📅 Año de publicación: 1965  
🌟 Puntuación: No disponible  

Justificación: "Dune" comparte con Star Wars elementos de narrativa épica, exploraciones filosóficas y un mundo futurista rico en detalles, aunque se centra más en temas de poder, religión y ecología.


In [55]:
def pelis_agent(state):
    llm = OllamaLLM(
        model="qwen2.5",
        base_url="http://localhost:11434",
        num_gpu_layers=90,
        num_ctx=1536,
        temperature=0.5,
    )
    prompt = PromptTemplate.from_template(
        """ Eres un critico de cine experto en recomendar libros a los usuarios basandote en sus preferencias.
            Tienes que dar una recomendación basandote en el input que recibas del usuario (recibiras un libro) que te diga el usuario 
            y teniendo en cuenta el contexto que tienes con la información de libros.
            
             
            Contexto: {context}
                                            
    
            Pregunta del usuario: {input}
                                                   
            Genera la recomendación utilizando la siguiente estructura
            
            📚 Pelicula recomendado: [Título exacto del libro]
            ✍️ Autor: [Nombre del autor] (si disponible) 
            📅 Año de publicación: [Año] (si disponible) 
            🌟 Puntuación: [X.X/5] (si disponible)   


            Justifica la recomendación con un breve parrafo. No te extiendas mas de 3 lineas.

            Justificación: [2-3 frases sobre similitudes con el libro de referencia]

            Reglas:
            1. Usa SOLO información de las peliculas proporcionados.
            2. Si no hay libros similares, di: "No encontré coincidencias precisas".
            3. Mantén la respuesta concisa y profesional.
        
        """
    )
    rag_chain = (
    {
        "context": itemgetter("input") | retrievermovies,
        "input": itemgetter("input"),
    }
    | prompt
    | llm
    | StrOutputParser()
)

    response = rag_chain.invoke({"input": state["input"]})
    return {"output": response}

In [31]:
# Agente general
def general_agent(state):
    llm = OllamaLLM(
    model="qwen2.5",
    base_url="http://localhost:11434",
    num_gpu_layers=90,       # Aceleración Metal máxima
    num_ctx=1536,            # Balance memoria/contexto
    temperature=0.5,
    )

    prompt = PromptTemplate.from_template(
        """Da una respuesta breve y concisa a la siguiente pregunta: {input}

        Si no sabes la respuesta di: "Con mis conocimientos no te puedo dar una respuesta a tu pregunta"

        """
    )
    chain = prompt | llm
    response = chain.invoke({"input": state["input"]})
    return {"output": response}

In [51]:
from langgraph.graph import StateGraph, END
#You can precise the format here which could be helpfull for multimodal graphs
class AgentState(TypedDict):
    input: str
    output: str
    decision: str
def create_graph():    
    workflow = StateGraph(AgentState)

    workflow.add_node("analyze", analyze_question)
    workflow.add_node("libros_agent", libros_agent)
    workflow.add_node("pelis_agent", pelis_agent)
    workflow.add_node("general_agent", general_agent)

    workflow.add_conditional_edges(
        "analyze",
        lambda x: x["decision"],
        {
            "libro": "libros_agent",
            "pelicula": "pelis_agent",
            "ninguna": "general_agent"
        }
    )

    workflow.set_entry_point("analyze")
    workflow.add_edge("general_agent", END)
    workflow.add_edge("libros_agent", END)
    workflow.add_edge("pelis_agent", END)

    return workflow.compile()

In [53]:
# Crea el grafo
app = create_graph()


In [ ]:
# Ejemplo 1: Pelicula
input_data = {
    "input": "Quiero un libro que sea similar a Star Wars",
    "output": "",
    "decision": ""   # Esta la llenará la función analyze_question
}
result1 = app.invoke({"input": "Quiero una pelicula que sea similar a Star Wars"})
print("Recomendacion:")
print(result1["output"])

Recomendacion:
📚 Pelicula recomendada: Blade Runner
🌟 Puntuación: No disponible

Justificación: Blade Runner comparte con Star Wars una atmósfera futurista y un tono más sombrío, además de explorar temas como la identidad y el control tecnológico.


In [64]:
# Ejemplo 2: Libro
result1 = app.invoke({"input": "Quiero un libro que se parezca a la cancion de Wonderwall de Oasis"})
print("Recomendacion:")
print(result1["output"])

Recomendacion:
No encontré coincidencias precisas.

Justificación: La canción "Wonderwall" de Oasis es una pieza musical que no tiene directamente un equivalente en los libros proporcionados, todos de temática distópica o ficción científica. Sin embargo, si buscas algo con temas profundos y narrativas complejas, te recomendaría "El nombre del viento", que cuenta la historia de un personaje con talentos excepcionales, similar a la emoción que puede generar una canción icónica.


In [62]:
# Ejemplo 2: Libro
result1 = app.invoke({"input": "que son los tornillos?"})
print("Recomendacion:")
print(result1["output"])

Recomendacion:
Los tornillos son piezas metálicas con un filo helicoidal que se utilizan para fijar y unir materiales aprietándolos en otras piezas previamente agujereadas.


In [9]:
# Crea el grafo
app = create_graph()

# Ejemplo 1: Pregunta matemática
result1 = app.invoke({"input": "Calcula la suma de 5 y 3"})
print("Resultado Matemáticas:")
print(result1["output"])
print("Herramientas usadas:", result1.get("used_tools", []))

# Ejemplo 2: Pregunta de lengua
result2 = app.invoke({"input": "Analiza esta oración gramatical, la oracion es la siguiente: Quiero jugar al futbol."})
print("\nResultado Lengua:")
print(result2["output"])

# Ejemplo 3: Pregunta general
result3 = app.invoke({"input": "Que es un tornillo"})
print("\nResultado General:")
print(result3["output"])

Resultado Matemáticas:
¡Claro! Vamos a resolver esta operación paso a paso:

### Paso 1: Identificar los números a sumar
Tenemos dos números que necesitamos sumar:
- El primer número es **5**.
- El segundo número es **3**.

### Paso 2: Aplicar la regla de suma
La suma se realiza al combinar o añadir estos dos valores. En este caso, estamos sumando un número entero positivo (5) y otro número entero positivo (3).

### Paso 3: Realizar la operación aritmética
Para sumar estos números, simplemente combinamos los valores:
\[ 5 + 3 = 8 \]

### Paso 4: Verificar el resultado
Podemos verificar que nuestro cálculo es correcto si consideramos una representación visual o numérica. Por ejemplo, si tenemos 5 objetos y añadimos 3 más, en total tendremos 8 objetos.

### Conclusion
La suma de 5 y 3 es **8**.

Entonces, la respuesta final a la pregunta "Calcula la suma de 5 y 3" es:
\[ \boxed{8} \]
Herramientas usadas: []

Resultado Lengua:
Claro, voy a analizar la oración "Quiero jugar al fútbol" paso